In [1]:
import basicLib.loadAndTest as orig
import basicLib.featureUpdate as featUp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import *
from pylab import *
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import re
from sklearn.linear_model import LogisticRegression
import basicLib.modelRun as runms
from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time
import clf2net
import clf2netTest
import clf2netTree
from sklearn import ensemble

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pathTrain = '../input_clean/users_train_final.csv'
pathTest = '../input_clean/users_test_final.csv'
pathyRes = '../input_clean/yRes_final.csv'

In [3]:
finalTrain = pd.read_csv(pathTrain)
finalTest = pd.read_csv(pathTest)
yRes = pd.read_csv(pathyRes)

In [4]:
finalTrainUse = finalTrain.drop('id',axis=1)
finalTestUse = finalTest.drop('id',axis=1)

### training

In [8]:
X_train, X_test, y_train, y_test = train_test_split(finalTrain, yRes['country_destination'])

In [ ]:
predictMethod = LogisticRegression()
predictMethod.fit(X_train,y_train)

In [ ]:
y_pred = predictMethod.predict(X_test)
accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod.predict_proba(X_test)

In [ ]:
accuracy_score

In [ ]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    myArr.append(predictMethod.classes_[order[:5]])

In [14]:
len(finalTrainUse)

213451

### send to airbnb - regression

In [5]:
start = time.time()
predictMethod = LogisticRegression()
predictMethod.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

401.3115849494934


In [6]:
y_pred = predictMethod.predict(finalTestUse)
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod.predict_proba(finalTestUse)

#### only 1 destionation

In [7]:
idCol = finalTest['id']
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}

In [8]:
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_regression_sessionsS.csv',index=False)

#### to put 5 destination

In [10]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in predictMethod.classes_[order[:5]]:
        myArr.append(i)
idCol = finalTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)

In [11]:
d = {'id' : idCol , 'country' : pd.Series(myArr)}

In [12]:
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_regression_sessionsM.csv',index=False)

### send to airbnb - tree

In [13]:
start = time.time()
predictMethod_tree = tree.DecisionTreeClassifier()
predictMethod_tree.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

18.91662073135376


In [14]:
y_pred = predictMethod_tree.predict(finalTestUse)
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod_tree.predict_proba(finalTestUse)

In [16]:
idCol = finalTest['id']
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}

In [18]:
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_tree_sessionsS.csv',index=False)

In [19]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in predictMethod.classes_[order[:5]]:
        myArr.append(i)
idCol = finalTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol , 'country' : pd.Series(myArr)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_tree_sessionsM.csv',index=False)

### send to airbnb - neural

In [20]:
start = time.time()
predictMethod_NN = MLPClassifier(solver='lbfgs', alpha=1e-5)
predictMethod_NN.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

179.21100521087646


In [21]:
y_pred = predictMethod_NN.predict(finalTestUse)
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod_NN.predict_proba(finalTestUse)
idCol = finalTest['id']
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_NN_sessionsS.csv',index=False)
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in predictMethod.classes_[order[:5]]:
        myArr.append(i)
idCol = finalTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol , 'country' : pd.Series(myArr)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_NN_sessionsM.csv',index=False)